In [1]:
import pandas as pd
import numpy as np
import nltk
import sys
import matplotlib.pyplot as plt
import json
sys.path.append("./torchMoji")

import os
import sys

sys.path.append('../')

import torch
import torch.utils.data

from torch import nn
import time
import json
import numpy as np
from torch import optim
from torch.nn import functional as F
from experiments.TorchHelper import TorchHelper
from experiments import utils as U
from models.model import *
import experiments.config as C
import shutil
from sklearn.metrics import f1_score
import warnings
import copy
from nltk.tokenize import TweetTokenizer
import random
from torchMoji.torchmoji.word_generator import WordGenerator
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
import ast
import numpy as np

### Try Torchmoji Vocab builder


In [ ]:
from torchMoji.torchmoji.word_generator import WordGenerator 

In [ ]:
word_gen = WordGenerator(stream=df_train, ignore_emojis=False, allow_unicode_text=True)

list_words = []
maxm = None
maxm_text = ""
for df in [df_train, df_dev]:
    for text in df["Text"]:
        if maxm is None:
            maxm = len(word_gen.get_words(text))
            maxm_text = text
        else:
            if len(word_gen.get_words(text)) > maxm:
                maxm = len(word_gen.get_words(text))
                maxm_text = text
        
        list_words += word_gen.get_words(text)

In [ ]:
maxm_text

In [ ]:
# Take the set of words from the corpus and build a vocab with key as the token and value as the index.
vocab_dict = {}
for idx, word in enumerate(set(list_words)):
    vocab_dict[word] = idx


In [ ]:
import json
json_object = json.dumps(vocab_dict, indent = 4)
with open("./preprocessing/vocab.json", "w", encoding ="utf-8") as file:
    file.write(json_object)

# EXPLORATORY DATA ANALYSIS

In [2]:
# Load the dataset

train_path = "./eng/trac2_eng_train.csv"
dev_path = "./eng/trac2_eng_dev.csv"

In [3]:
df_train = pd.read_csv(train_path)
df_dev = pd.read_csv(dev_path)

In [4]:
df_train.head()

,ID,Text,Sub-task A,Sub-task B
0,C45.451,Next part,NAG,NGEN
1,C47.11,Iii8mllllllm\nMdxfvb8o90lplppi0005,NAG,NGEN
2,C33.79,🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...,NAG,NGEN
3,C4.1961,What the fuck was this? I respect shwetabh and...,NAG,NGEN
4,C10.153,Concerned authorities should bring arundathi R...,NAG,NGEN


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4263 entries, 0 to 4262
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          4263 non-null   object
 1   Text        4263 non-null   object
 2   Sub-task A  4263 non-null   object
 3   Sub-task B  4263 non-null   object
dtypes: object(4)
memory usage: 133.3+ KB


In [ ]:
def gen_label_dist(df):
    label_vals = []
    labels = []
    for taskA in df["Sub-task A"].unique():
        for taskB in df["Sub-task B"].unique():
            labels.append(f"{taskA}-{taskB}")
            label_vals.append(len(df[(df["Sub-task A"] == taskA) & (df["Sub-task B"] == taskB)]))
    
    dist_dict = dict(zip(labels, label_vals))
    print(dist_dict)
    
    df_labels_dist = pd.DataFrame({"Label": labels, "Frequency": label_vals})
    
    return df_labels_dist


### Training Data Label wise Distribution 

In [ ]:
df_train_dist = gen_label_dist(df_train)
df_train_dist.plot.bar(x='Label', y='Frequency', rot=0)
plt.show()

### Dev Data Label wise Distribution

In [ ]:
df_dev_dist = gen_label_dist(df_dev)
df_dev_dist.plot.bar(x='Label', y='Frequency', rot=0, color='red')
plt.show()

Clearly as we can see, there is data skewness for the datapoints of NAG-NGEN. The case of interest for us are CAG-GEN, OAG-GEN and NAG-GEN.

## Manually Inspect the Gender Aggression Text 

In [ ]:
# OAG - GEN
# TRAIN
train_oag_gen = df_train[(df_train["Sub-task A"] == "OAG") & (df_train["Sub-task B"] == "GEN")]["Text"]
train_oag_gen.to_csv("train_oag_gen.csv")
print(len(train_oag_gen))

# DEV
dev_oag_gen = df_dev[(df_dev["Sub-task A"] == "OAG") & (df_dev["Sub-task B"] == "GEN")]["Text"]
dev_oag_gen.to_csv("dev_oag_gen.csv")
print(len(dev_oag_gen))

In [ ]:
# CAG - GEN
# TRAIN
train_cag_gen = df_train[(df_train["Sub-task A"] == "CAG") & (df_train["Sub-task B"] == "GEN")]["Text"]
train_cag_gen.to_csv("train_cag_gen.csv")
print(len(train_cag_gen))

# DEV
dev_cag_gen = df_dev[(df_dev["Sub-task A"] == "CAG") & (df_dev["Sub-task B"] == "GEN")]["Text"]
dev_cag_gen.to_csv("dev_cag_gen.csv")
print(len(dev_cag_gen))

In [ ]:
# CAG - GEN
# TRAIN
train_nag_gen = df_train[(df_train["Sub-task A"] == "NAG") & (df_train["Sub-task B"] == "GEN")]["Text"]
train_nag_gen.to_csv("train_nag_gen.csv")
print(len(train_nag_gen))

# DEV
dev_nag_gen = df_dev[(df_dev["Sub-task A"] == "NAG") & (df_dev["Sub-task B"] == "GEN")]["Text"]
dev_nag_gen.to_csv("dev_nag_gen.csv")
print(len(dev_nag_gen))

### Review and label the train and dev {OAG, CAG, NAG} - GEN texts 

* 1 - Misogyny 

* 0 - Neutral stance

* Code switching 

* Code mixing

There are typos also to be observed in some of the texts, as can be seen below : 

Train-Set


* `All these urban naxaulites like Arundhati Roy have citizenship in other countries, My hindu brothers, where will u go? Fir shake of money they will sell us! What this Prustitute Arundhati Roy did for the nation's economy, science, technology, envirniment, poor...nothing!`


There are cases where the person is making derogatory remark over wrong behaviour of women using strong words, should that be a genuine hate speech text or not is tough call to make. For example


* `What's even more disgusting is that not 1 INDIAN WOMAN has the courage to post
a comment against this....probably they fear the backlash....WHAT A SHAME ON
SOCIETY!!!!! Fuck off you BITCHES!!!!!!!!!!!!!!!!!!!!!!!`


* We will get the output for the code mixed and code switched gendered aggression cases too and see, how the model performs.


* The Text which truly represents Female Gendered Aggression filtered out after manual review. Most of the textual artefacts are code switched and mixed. Would be interesting to see how the misogyny detection model performs over the same, but the issue would come in finding the sentiment values of those code switched words in the MESS computation. We will see, how to figure this out.


* Clear Definition of the Misogyny is required as it can be seen in the dataset that Gendered aggressive comments targets male population too. Another point to be observed in the dataset, it has a very localised data collection which involves people from India commenting on the bollywood's feminism problem, or targetting politician for their double standard stances. It doesnot represent a global view of the misogynistic behavior.





In [ ]:
# Import reviwed Gen Aggression Files.
df_rev_oag_gen_tr = pd.read_csv("./reviewed/train_oag_gen.csv")
df_rev_cag_gen_tr = pd.read_csv("./reviewed/train_cag_gen.csv")
df_rev_oag_gen_dev = pd.read_csv("./reviewed/dev_oag_gen.csv")
df_rev_cag_gen_dev = pd.read_csv("./reviewed/dev_cag_gen.csv")

list_rev_df = [
    df_rev_oag_gen_tr, 
    df_rev_cag_gen_tr, 
    df_rev_oag_gen_dev, 
    df_rev_cag_gen_dev
]

In [ ]:
list_cnt = []
list_col_name = ["rev_oag_gen_tr", "rev_cag_gen_tr", "rev_oag_gen_dev", "rev_cag_gen_dev"]

for df_rev in list_rev_df:
    if "Comment" in df_rev.columns:
        list_cnt.append(len(df_rev[df_rev["Comment"] == '1']))
    if "Comments" in df_rev.columns:
        list_cnt.append(len(df_rev[df_rev["Comments"] == 1]))

df_rev_plot = pd.DataFrame({"Rev-Data": list_col_name, "Frequency": list_cnt})

In [ ]:
df_rev_plot.plot.bar(x='Rev-Data', y='Frequency', rot=0, color='red')
plt.show()
df_rev_plot

` SOME OF THE STATS AFTER THE REVIEW `


Before Review :

OAG-GEN(Train) = 140

OAG-GEN(Dev) = 26


After Review:

OAG-GEN(Train) = 44

OAG-GEN(Dev) = 7


Before Review :

CAG-GEN(Train) = 35

CAG-GEN(Dev) = 9


Before Review :

CAG-GEN(Train) = 4

CAG-GEN(Dev) = 2

###  Get the Model for the Inference 

In [7]:
def load_model():
    """
    Creates and returns the model.
    Moves to GPU if found any.
    :return:

    """

    model = BERTRA()

    torch_helper = TorchHelper() 
    
    # /Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/weights/pretrained_models/eng_model_best.pth
    dir_model = "../weights/pretrained_models/eng_model_best.pth"
    
    torch_helper.load_saved_model(model, dir_model)
    print('model loaded')
    
    return model

model = load_model()

model loaded


In [8]:
def pad_features(docs_ints, seq_length=200):

    # getting the correct rows x cols shape
    features = np.zeros((len(docs_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(docs_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

def masking(docs_ints, seq_length=200):

    # getting the correct rows x cols shape
    masks = np.zeros((len(docs_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(docs_ints):
        #mask[i, :len(row)] = 1
        masks[i, -len(row):] = 1

    return masks


# Load the preprocessed JSON.
features_dev = json.load(open('./preprocessed_data/eng_dev.json'))

dev_set = [val for key,val in features_dev.items()]
print('Validation Loaded')

test_set = dev_set[0:10]

Validation Loaded


In [9]:

agg_labels = {0 : "NAG", 1: "OAG", 2: "CAG"}
gen_labels ={0: "NGen", 1:"Gen"}


def pred(model, test_df, agg_dict, gen_dict):
    
    batch_x, batch_t, batch_y1, batch_y2 = [], [], [], []

    for i in range(len(test_df)):
        batch_x.append(test_df[i]['tokenized'])
        batch_t.append(test_df[i]['DM'])

        batch_y1.append(test_df[i]['y1'])
        batch_y2.append(test_df[i]['y2'])

    mask = masking(batch_x)
    padded = pad_features(batch_x)

    out = model.forward(torch.tensor(padded), torch.tensor(mask))

    y_pred1 = out['y_pred1'].cpu()
    y_pred2 = out['y_pred2'].cpu()
    
    pred_agg_labels = [agg_dict[val] for val in np.argmax(y_pred1.detach().numpy(), axis=1)]
    pred_gen_labels = [gen_dict[val] for val in np.argmax(y_pred2.detach().numpy(), axis=1)]
    
    # Return DataFrame with 
    # Columns 1: Text, Column 2: Label-1, Column 3: Label-2
    df_pred = pd.DataFrame({
        "Text": [test_df[i]['post'] for i in range(len(test_df))],
        "Label-1": pred_agg_labels,
        "Label-2": pred_gen_labels
                           })
    
    return df_pred


pred(model, test_set, agg_labels, gen_labels)


,Text,Label-1,Label-2
0,U deserve more subscribers. U really great.,NAG,NGen
1,Nice video....,NAG,NGen
2,sorry if i bother somebody.. iam a defence asp...,NAG,Gen
3,Joker was amazing....it was not glamorised !.....,NAG,NGen
4,Nice baro,NAG,NGen
5,she telling to give the fake information it sh...,NAG,NGen
6,"@username c whateva u said,is true........thes...",CAG,NGen
7,"Love your review bhai , and movie awesome hai",NAG,NGen
8,Randi Naxalist Roy,NAG,NGen
9,I hate ranu mondal💀☠️👿👿👿,OAG,NGen


### Pointers to Observe for the model training:

* The tokenized key -> comes from the BERT tokenizer
* DM -> The token ids from our vocab generated using custom built vocab.
* DM is not used for the inference neither, it is used in the training.
* The author never used a custom built vocab for the purpose.


## Sub Filter the Train and Dev Test based on the Label Space and Model Prediction

### OAG-GEN-DEV SET


In [10]:
# Output Prediction for the Datapoint where the manual review declared it as Actually aggressive and Gendered

dev_set_feat = [val for key,val in features_dev.items()]


def filter_feat(ft_list, df):
    list_ft = []
    for text in df["Text"]:
        for ft in ft_list:
            if ft["post"] == text:
                list_ft.append(ft)
                break
    
    return list_ft

# filter the datapoint which were actually aggressive and genered.
df_oag_filter = df_rev_oag_gen_dev[df_rev_oag_gen_dev["Comments"] == 1]

# Convert the above filtered set into model features ...
test_set = filter_feat(dev_set_feat, df_oag_filter)

# Perform the prediction on top of the model ... 
pred(model, test_set, agg_labels, gen_labels)

NameError: name 'df_rev_oag_gen_dev' is not defined

### CAG-GEN-DEV SET

In [ ]:
df_rev_cag_gen_dev
dev_set = [val for key,val in features_dev.items()]


def filter_feat(ft_list, df):
    list_ft = []
    for text in df["Text"]:
        for ft in ft_list:
            if ft["post"] == text:
                list_ft.append(ft)
                break
    
    return list_ft

df_rev_dev = df_rev_cag_gen_dev[df_rev_cag_gen_dev["Comments"] == 1]

test_set = filter_feat(dev_set, df_rev_cag_gen_dev)
pred(model, test_set, agg_labels, gen_labels)

# -------------------------------------------------------------------------------------------

# Data Analysis

#### Flow

1. Take the dev set. We will move to the train later, first check on the datapoints where the distribution is unseen. 
2. Filter on the NAG, OAG, CAG and Gen-NonGen.
3. First Preference - Gendered + {NAG, OAG, CAG}.
4. Perform the Inference on top of the set and filter the exact match and Non-Match.
5. Exact Match are the cases where the model is doing exactly as per expected. -> Here we can also check the probability value for that particular label. How confident the model seems.
6. Take the above set and generate the Explanation using HEDGE.
7. Take the output of the explanation weights and the sentiment values from the sentiment vocab -> generate MESS.
8. Now go back and take up the mismatch set, where the label gave wrong prediction and then perform the steps 5 -7.
9. If the explanation model generate high weight emphasis for the correct set of words but gives a wrong label, this means model is performing some shortcut learning cues inside to be able to come-up with actual prediction. - We will have to see.. But this case is interesting from the perspective... did model give wrong label for the right reasons for wrong label bcos of the wrong reasons.. again subject to the explanation algorithm that we are using.

In [ ]:
# Import the Dev Set

dev_path = "eng/trac2_eng_dev.csv"
df_dev = pd.read_csv(dev_path)

In [ ]:
df_dev

In [ ]:
# Segregate OAG-GEN, CAG-GEN and NAG-GEN
dev_OG = df_dev[(df_dev["Sub-task A"] == "OAG") & (df_dev["Sub-task B"] == "GEN")]
dev_CG = df_dev[(df_dev["Sub-task A"] == "CAG") & (df_dev["Sub-task B"] == "GEN")]
dev_NG = df_dev[(df_dev["Sub-task A"] == "NAG") & (df_dev["Sub-task B"] == "GEN")]


# Segregate OAG-NGEN, CAG-NGEN, NAG-NGEN
dev_ONG = df_dev[(df_dev["Sub-task A"] == "OAG") & (df_dev["Sub-task B"] == "NGEN")]
dev_CNG = df_dev[(df_dev["Sub-task A"] == "CAG") & (df_dev["Sub-task B"] == "NGEN")]
dev_NNG = df_dev[(df_dev["Sub-task A"] == "NAG") & (df_dev["Sub-task B"] == "NGEN")]


In [ ]:
agg_labels = {0 : "NAG", 1: "OAG", 2: "CAG"}
gen_labels = {0: "NGEN", 1:"GEN"}


def pred(model, test_df, agg_dict, gen_dict):
    
    batch_x, batch_t, batch_y1, batch_y2 = [], [], [], []

    for i in range(len(test_df)):
        batch_x.append(test_df[i]['tokenized'])
        batch_t.append(test_df[i]['DM'])

        batch_y1.append(test_df[i]['y1'])
        batch_y2.append(test_df[i]['y2'])

    mask = masking(batch_x)
    padded = pad_features(batch_x)

    out = model.forward(torch.tensor(padded), torch.tensor(mask))

    y_pred1 = out['y_pred1'].cpu()
    y_pred2 = out['y_pred2'].cpu()
    
    pred_agg_prob = [val for val in y_pred1.detach().numpy()]
    pred_gen_prob = [val for val in y_pred2.detach().numpy()]
    
    pred_agg_labels = [agg_dict[val] for val in np.argmax(y_pred1.detach().numpy(), axis=1)]
    pred_gen_labels = [gen_dict[val] for val in np.argmax(y_pred2.detach().numpy(), axis=1)]
    
    
    # Return DataFrame with 
    # Columns 1: Text, Column 2: Label-1, Column 3: Label-2
    df_pred = pd.DataFrame({
        "Text": [test_df[i]['post'] for i in range(len(test_df))],
        "Label-1": pred_agg_labels,
        "Label-2": pred_gen_labels,
        "Agg-Prob": pred_agg_prob,
        "Gen-Prob": pred_gen_prob
                           })
    
    return df_pred


def filter_feat(ft_list, df):
    list_ft = []
    for text in df["Text"]:
        for ft in ft_list:
            if ft["post"] == text:
                list_ft.append(ft)
                break
    
    return list_ft


def pad_features(docs_ints, seq_length=200):

    # getting the correct rows x cols shape
    features = np.zeros((len(docs_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(docs_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

def masking(docs_ints, seq_length=200):

    # getting the correct rows x cols shape
    masks = np.zeros((len(docs_ints), seq_length), dtype=int)

    # for each review, I grab that review and
    for i, row in enumerate(docs_ints):
        #mask[i, :len(row)] = 1
        masks[i, -len(row):] = 1

    return masks

def load_model():
    """
    Creates and returns the model.
    Moves to GPU if found any.
    :return:

    """

    model = BERTRA()

    torch_helper = TorchHelper()
    
    # /Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/weights/pretrained_models/eng_model_best.pth
    dir_model = "../weights/pretrained_models/eng_model_best.pth"
    
    torch_helper.load_saved_model(model, dir_model)
    print('Model loaded')
    
    return model


def filterMatchPred(df_gt, df_pred):
    gt_list = []
    for text in df_pred["Text"]:
        gt_label = df_gt[df_gt["Text"] == text]["GT-Label"].values[0]
        gt_list.append(gt_label)
    
    df_pred["GT-Label"] = gt_list
    
    # Instances where, there is exact match of the pred and GT.
    # np.where(df["col1"] == df["col2"], True, False)

    idxs = df_pred["Pred-Label"].eq(df_pred["GT-Label"])
    
    df_filter = df_pred[idxs]

    return df_pred, df_filter

### Misogyny Detection Inference over DEV-OAG-GEN

In [12]:
# Generate Prediction based on the

# Load the Feature Set of the Dev Set 
# Load the preprocessed JSON.
features_dev = json.load(open('./preprocessed_data/eng_dev.json'))
dev_ft = [val for key,val in features_dev.items()]

# Load the Misogyny Detection Model
model = load_model()

test_set = filter_feat(dev_ft, dev_OG)
pred_dev_OG = pred(model, test_set, agg_labels, gen_labels)

# Concatentate the Label-1 and Label-2 Together
pred_dev_OG["Pred-Label"] = pred_dev_OG["Label-1"] + "-" + pred_dev_OG["Label-2"]
pred_dev_OG.drop(["Label-1", "Label-2"], axis=1, inplace=True)
pred_dev_OG = pred_dev_OG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]
pred_dev_OG.info()

pred_dev_OG.head()

model loaded


NameError: name 'dev_OG' is not defined

### Misogyny Detection Inference over DEV-CAG-GEN

In [ ]:
test_set = filter_feat(dev_ft, dev_CG)
pred_dev_CG = pred(model, test_set, agg_labels, gen_labels)
# Concatentate the Label-1 and Label-2 Together
pred_dev_CG["Pred-Label"] = pred_dev_CG["Label-1"] + "-" + pred_dev_CG["Label-2"]
pred_dev_CG.drop(["Label-1", "Label-2"], axis=1, inplace=True)
pred_dev_CG = pred_dev_CG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]
pred_dev_CG.info()
pred_dev_CG.head()

### Misogyny Detection Inference over DEV-NAG-GEN

In [ ]:
test_set = filter_feat(dev_ft, dev_NG)
pred_dev_NG = pred(model, test_set, agg_labels, gen_labels)

# Concatentate the Label-1 and Label-2 Together
pred_dev_NG["Pred-Label"] = pred_dev_NG["Label-1"] + "-" + pred_dev_NG["Label-2"]
pred_dev_NG.drop(["Label-1", "Label-2"], axis=1, inplace=True)
pred_dev_NG = pred_dev_NG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]
pred_dev_NG.info()
pred_dev_NG.head()

### Filter Prediction Matching with GT and Not Matching with GT

In [ ]:
# df_dev set -> GT Label made consistent with the Predicted Label
if "Sub-task A" in df_dev:
    df_dev["GT-Label"] = df_dev["Sub-task A"] + "-" + df_dev["Sub-task B"]
    df_dev.drop(["Sub-task A", "Sub-task B"], axis=1, inplace=True)


In [ ]:
df_dev

In [ ]:
# Filter OG
df_pred_OG, df_filter_OG = filterMatchPred(df_dev, pred_dev_OG)

In [ ]:
print(len(df_pred_OG))
print(len(df_filter_OG))

In [ ]:
# Filter CG
df_pred_CG, df_filter_CG = filterMatchPred(df_dev, pred_dev_CG)
print(len(df_pred_CG))
print(df_filter_CG)

In [ ]:
# Filter NG
df_pred_NG, df_filter_NG = filterMatchPred(df_dev, pred_dev_NG)
print(len(df_pred_NG))
print(len(df_filter_NG))

**Match Statistics for the Gendered Dev Set**

Out of total 22 OG instances - 1 Matches - 4.5% Accuracy

Out of total 9 CG instances - 0 Matches - 0 % Accuracy

Out of total 34 NG instances - 5 Matches - 5 % Accuracy

### Misogyny Detection Inference over DEV-OAG-NGEN

In [ ]:
test_set = filter_feat(dev_ft, dev_ONG)
pred_dev_ONG = pred(model, test_set, agg_labels, gen_labels)

# Concatentate the Label-1 and Label-2 Together
pred_dev_ONG["Pred-Label"] = pred_dev_ONG["Label-1"] + "-" + pred_dev_ONG["Label-2"]
pred_dev_ONG.drop(["Label-1", "Label-2"], axis=1, inplace=True)
pred_dev_ONG = pred_dev_ONG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]
pred_dev_ONG.info()

pred_dev_ONG.head()

### Misogyny Detection Inference over DEV-CAG-NGEN

In [ ]:
test_set = filter_feat(dev_ft, dev_CNG)
pred_dev_CNG = pred(model, test_set, agg_labels, gen_labels)

# Concatentate the Label-1 and Label-2 Together
pred_dev_CNG["Pred-Label"] = pred_dev_CNG["Label-1"] + "-" + pred_dev_CNG["Label-2"]
pred_dev_CNG.drop(["Label-1", "Label-2"], axis=1, inplace=True)
pred_dev_CNG = pred_dev_CNG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]
pred_dev_CNG.info()
pred_dev_CNG.head()

### Misogyny Detection Inference over DEV-NAG-NGEN

In [ ]:
test_set = filter_feat(dev_ft, dev_NNG)

df_pred_list = []
total_ele = 0 

for idx in range(8):

    if idx < 7:
        test_sub = test_set[idx*100: (idx+1)*100]
    else:
        test_sub = test_set[idx*100: ]
    
    pred_dev_NNG_sub = pred(model, test_sub, agg_labels, gen_labels)

    # Concatentate the Label-1 and Label-2 Together
    pred_dev_NNG_sub["Pred-Label"] = pred_dev_NNG_sub["Label-1"] + "-" + pred_dev_NNG_sub["Label-2"]
    pred_dev_NNG_sub.drop(["Label-1", "Label-2"], axis=1, inplace=True)
    
    df_pred_list.append(pred_dev_NNG_sub)
    total_ele += len(pred_dev_NNG_sub)

pred_dev_NNG = pd.concat(df_pred_list)
pred_dev_NNG = pred_dev_NNG[["Text", "Pred-Label", "Agg-Prob", "Gen-Prob"]]


In [11]:
if 0:
    pred_dev_ONG.to_csv("pred_dev_ONG.csv")
    pred_dev_CNG.to_csv("pred_dev_CNG.csv")
    pred_dev_NNG.to_csv("pred_dev_NNG.csv")
    pred_dev_OG.to_csv("pred_dev_OG.csv")
    pred_dev_CG.to_csv("pred_dev_CG.csv")
    pred_dev_NG.to_csv("pred_dev_NG.csv")

In [26]:
x = pd.read_csv("./model_outputs/dev-prediction/pred_dev_NG.csv")
print(len(x))
len(x[x["Pred-Label"] == x["GT-Label"]])


34


5

In [ ]:
print(pred_dev_NNG.info())
pred_dev_NNG.head()

In [ ]:
pred_dev_NNG.reset_index(drop=True, inplace=True)

### Filter Prediction Matching with GT and Not Matching with GT

In [ ]:
# Filter ONG
df_pred_ONG, df_filter_ONG = filterMatchPred(df_dev, pred_dev_ONG)

# Filter CNG
df_pred_CNG, df_filter_CNG = filterMatchPred(df_dev, pred_dev_CNG)

# Filter NNG
df_pred_NNG, df_filter_NNG = filterMatchPred(df_dev, pred_dev_NNG)


In [ ]:


# Store the CSVs
if 0:
    pred_dev_ONG.to_csv("./model_outputs/dev-prediction/pred_dev_ONG.csv")
    pred_dev_CNG.to_csv("./model_outputs/dev-prediction/pred_dev_CNG.csv")
    pred_dev_NNG.to_csv("./model_outputs/dev-prediction/pred_dev_NNG.csv")
    pred_dev_OG.to_csv("./model_outputs/dev-prediction/pred_dev_OG.csv")
    pred_dev_CG.to_csv("./model_outputs/dev-prediction/pred_dev_CG.csv")
    pred_dev_NG.to_csv("./model_outputs/dev-prediction/pred_dev_NG.csv")

# Store filtered DataFrames
if 0:
    df_filter_CNG.to_csv("./model_outputs/filt-dev-pred/filter_CNG.csv")
    df_filter_ONG.to_csv("./model_outputs/filt-dev-pred/filter_ONG.csv")
    df_filter_NNG.to_csv("./model_outputs/filt-dev-pred/filter_NNG.csv")
    df_filter_CG.to_csv("./model_outputs/filt-dev-pred/filter_CG.csv")
    df_filter_OG.to_csv("./model_outputs/filt-dev-pred/filter_OG.csv")
    df_filter_NG.to_csv("./model_outputs/filt-dev-pred/filter_NG.csv")


if 0:    
    # Formate the Filtered DataFrames to input to the Hedge Model
    names = ["filt_ONG", "filt_CNG", "filt_NNG", "filt_OG", "filt_CG", "filt_NG"]

    for i, test in enumerate([df_filter_ONG, df_filter_CNG, df_filter_NNG, df_filter_OG, df_filter_CG, df_filter_NG]):

        test = test[["Text", "Pred-Label"]]
        test.rename(columns={"Text": "sentence", "Pred-Label":"label"}, inplace=True)
        test.reset_index(drop=True, inplace=True)

        for idx in range(len(test)):
            test["sentence"][idx] = test["sentence"][idx].replace('\n', '')
            if "NGEN" in test["label"][idx]:
                test["label"][idx] = 0

            else:
                test["label"][idx] = 1

        test.to_csv(os.path.join("./model_outputs/dev-format-pred", names[i] +".tsv"), sep="\t", index=False)


**The Statistics for the Non Gendered Subsets over the Dev Set**

ONG : 7 : 77 -> 9% Acc

CNG : 4 : 97 -> 4% Acc

NNG : 616 : 714 -> 86% Acc

In [ ]:
train_path = "eng/trac2_eng_train.csv"
df_train = pd.read_csv(train_path)

In [ ]:
if "Sub-task A" in df_train.columns:
    df_train["GT-Label"] = df_train["Sub-task A"] + "-" + df_train["Sub-task B"]
    df_train.drop(["Sub-task A", "Sub-task B"], axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
fig, ax = plt.subplots()
ax = df_dev["GT-Label"].value_counts().plot(kind='bar', ax=ax)
ax.tick_params(axis='x', which='major', labelsize=7, rotation=0)
fig.savefig("fgs.png")


## Next Steps


1. Create a function which connect with the Hedge Algorithm and extracts the explanation scores.

2. For the cases where we have the exact Match of GT  with Pred - Generate the Explanation - Gendered
More Gendered Case, the important factor is to generate filter Female gender - Since that contributes to misogyny.

3. Then Generate the above results for the NGEN category of Datapoints where the GT matches with Pred

4. For the cases where the model output doesnot match with GT, check the probability values.. that how off the model is from the actual label in terms of the probability value.

5. Pointer 3 needs to be done overall for all the datapoints. w.r.t to the GT class. Per class how confident the model is at an average. GEN, NGEN class -> how confident the model is distribution of the all three label scores per datapoint in the task-1 and task-2.

6. Visualization - Correlation Sentiment with Explanation sentences.



# -----------------------------------------------------------------------------------------

In [ ]:
# Break down the Text into sub text and then feed it to the HEDGE system for explanation.
path = "/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-format-pred/filt_NNG.tsv"

df = pd.read_csv(path, sep="\t")

df["sentence"][31]

In [ ]:
list_sent = [
    "U deserve more subscribers. U really great.",
    "Joker was amazing.it was not glamorised but Kabir Singh was tatti 100X",
    "Love your review bhai , and movie awesome hai",
    "She must be sent to jail for anti national activities under NSA and PSA",
    "Excellent job...if law can't give you Justice....take it by yourself",
    "I'm also a lesbian girl and I'm proud of myself",
    "Arundhati's hairstyle is inspired by Dr. Mashahoor Gulati.",
    "Arrested for what? Know a little about our Constitution. Let's see who will arrest her.",
    "Please do a review on joker",
    "That's her pick up line that nobody listens",
    "Most honest and elaborate review thank u sir from GUJARAT. JAI SHRI RAM",
    "very well done salute to this guy",
    "Sir please upload the names of books which you read to develop mindset",
    "thankyou for the quote that u put back there it helped me."
]

In [ ]:
df = pd.DataFrame()
df["sentence"] = list_sent
df["label"] = [0 for _ in range(14)]
df

In [ ]:
df.to_csv(path, sep='\t', index=False)

**Pointer for the document - 11/05/2022**

1. Dev Set Statistics and train set Statistics
2. Show that Dev set prediction has higher number of data points being predicted of NG class. Reason being - Data imbalance in the train data.
3. Generate a plot of the prediction probability values for the class OG, CG, NG and ONG, CNG, and NNG. and check how much the model was confident for the GT class compared to the other classe. Indicates the confusion or clarity of the decision making of the model.
4. Take the Filtered out cases from the (OG, CG, NG) and (ONG, CNG, NNG). - Take about the format transition to run the HEDGE model.
5. Break down these sentence into more atomic sentences - reason being the computational intractability of the HEDGE run.
6. Generate the HEDGE explanation score for each of the sentence in the subsets. 
7. Manually review all the datapoints - Plot 2 examples from each of the subsets consistent with the explanation and anomalous in the explanation.

-------------------------------



# Misogyny and Aggression Model Inference Analysis

Here we will check how much clarity the model has in predicting the right class, more the difference between prediction probabilities and towards the right class, better clarity the model has in terms of the 
prediction made.

## Aggression Inference Analysis - Matching Classes

In [27]:
# Load the inference files
df_OG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_OG.csv")
df_CG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_CG.csv")
df_NG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_NG.csv")
df_ONG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_ONG.csv")
df_CNG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_CNG.csv")
df_NNG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_NNG.csv")


In [162]:
def predAnalyseOvertly(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[(df["Pred-Label"].apply(lambda x : "OAG" in x)) & (df["Pred-Label"] == df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[1] - probs[2])
        list_diffs.append(probs[1] - probs[0])
        
    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y3 = [val[2] for val in list_scores]
    
    width = 0.25

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2.5, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + width+0.1, y3, width=width, label='Covertly')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Overtly-Match.png")
        
    return list_scores, list_diffs
    

In [163]:
x, y = predAnalyseOvertly(df_OG)
print(x)
print(y)
np.mean(y)

[[0.03328269, 0.78107005, 0.1856473]]
[0.59542275, 0.74778736]


0.671605055

**As clearly can be seen, the mean difference between the Probabilities for the Overtly class, other Covertly class and Non Aggression class is 0.67. It is quite a high number to claim that the decision making for the model among the other classes is way less ambiguous.**

In [164]:
def predAnalyseCovertly(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "CAG" in x) & (df["Pred-Label"] == df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[2] - probs[1])
        list_diffs.append(probs[2] - probs[0])
    
    
    # Plot CO
    x = np.arange(len(list_scores))
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y3 = [val[2] for val in list_scores]
    width = 0.25

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + width/2, y2, width=width, label='Covertly')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Covertly-Match.png")
    
    
    return list_scores, list_diffs

In [165]:
x, y = predAnalyseCovertly(df_CG)
print(x)
print(y)
np.mean(y)

[]
[]


nan

**There were no data points for which the output of the misogyny aggression matches with the GT**

In [166]:
def predAnalyseNoagg(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "NAG" in x)& (df["Pred-Label"] == df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[0] - probs[1])
        list_diffs.append(probs[0] - probs[2])
        
        
    # Plot NO
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y3 = [val[2] for val in list_scores]
    
    width = 0.25

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + width+0.1, y3, width=width, label='Covertly')
    
    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/NonAgg-Match.png")    
    
    return list_scores, list_diffs

In [167]:
x, y = predAnalyseNoagg(df_NG)
print(x)
print(y)
np.mean(y)

[[0.81337935, 0.10682596, 0.07979469], [0.7449015, 0.08207034, 0.17302823], [0.55568665, 0.2614945, 0.18281882], [0.83159167, 0.08601627, 0.08239194], [0.7066269, 0.11711952, 0.1762536]]
[0.70655339, 0.73358466, 0.66283116, 0.57187327, 0.29419214999999993, 0.37286782999999996, 0.7455754, 0.74919973, 0.5895073799999999, 0.5303732999999999]


0.5956558269999999

## Aggression Inference Analysis - Non Matching Classes

In [168]:
def predAnalyseOvertly(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "OAG" not in x) & (df["Pred-Label"] != df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[1] - probs[2])
        list_diffs.append(probs[1] - probs[0])

    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y3 = [val[2] for val in list_scores]
    
    width = 0.25

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + width+0.1, y3, width=width, label='Covertly')
    
    ax.set_ylabel('Probabilites')
    ax.set_xticks(x)
    ax.set_title('Probabilities Comparison Classes')
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Overtly-NonMatch.png")

    
    return list_scores, list_diffs

In [169]:
x, y = predAnalyseOvertly(df_OG)
print(x)
print(y)
np.mean(y)

[[0.8843626, 0.02343824, 0.09219919], [0.4688446, 0.11857922, 0.4125761], [0.9919593, 0.00197632, 0.00606437], [0.69011015, 0.18602732, 0.12386259], [0.98343223, 0.00516575, 0.01140205], [0.9189583, 0.02567236, 0.05536939], [0.5589883, 0.15123467, 0.28977704], [0.92511237, 0.03537672, 0.0395109], [0.6387676, 0.29541448, 0.06581795], [0.71394944, 0.10214195, 0.18390869], [0.09192161, 0.44985652, 0.45822185], [0.98096746, 0.00976037, 0.00927225], [0.46854386, 0.35010877, 0.18134733], [0.9915154, 0.00405018, 0.0044344], [0.90520763, 0.06229217, 0.03250017], [0.98266923, 0.00351752, 0.01381334], [0.8894454, 0.04938887, 0.06116574], [0.6378985, 0.16123861, 0.20086282], [0.75712156, 0.12016895, 0.12270953]]
[-0.06876095, -0.86092436, -0.29399688, -0.35026538, -0.00408805, -0.98998298, 0.06216473, -0.50408283, -0.0062363, -0.97826648, -0.029697029999999996, -0.8932859400000001, -0.13854237, -0.40775363, -0.004134180000000001, -0.88973565, 0.22959652999999997, -0.34335312, -0.08176674000000002

-0.32983613842105264

In [28]:
def predAnalyseCovertly(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "CAG" not in x) & (df["Pred-Label"] != df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[2] - probs[1])
        list_diffs.append(probs[2] - probs[0])
    
    # Plot NO
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y2 = [val[2] for val in list_scores]
    
    width = 0.25

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + width+0.1, y2, width=width, label='Covertly')
    
    ax.set_ylabel('Probabilites')
    ax.set_xticks(x)
    ax.set_title('Probabilities Comparison Classes')
    ax.legend()

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Covertly-NonMatch.png")
    
    return list_scores, list_diffs

In [29]:
x, y = predAnalyseCovertly(df_CG)
print(x)
print(y)
np.mean(y)

[[0.29887846, 0.3957506, 0.30537093], [0.4079646, 0.36466235, 0.22737305], [0.89669734, 0.03689234, 0.06641035], [0.7931357, 0.16030382, 0.04656041], [0.94362795, 0.02596619, 0.03040579], [0.59996796, 0.1951777, 0.20485431], [0.49739638, 0.16845286, 0.3341508], [0.878064, 0.06312154, 0.05881455], [0.9016908, 0.05072821, 0.04758093]]
[-0.09037967000000002, 0.006492469999999972, -0.13728930000000003, -0.18059155000000002, 0.029518010000000004, -0.8302869900000001, -0.11374341000000002, -0.74657529, 0.004439599999999998, -0.91322216, 0.009676610000000002, -0.39511365, 0.16569794000000002, -0.16324558, -0.004306990000000004, -0.8192494499999999, -0.0031472800000000023, -0.85410987]


-0.2797464755555556

In [33]:
def predAnalyseNoagg(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    #corr_df = df[df["Pred-Label"].apply(lambda x : "NAG" not in x) & (df["Pred-Label"] != df["GT-Label"])]
    corr_df = df[df["Pred-Label"].apply(lambda x : "NAG" not in x)]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Agg-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1], probs[2]])
        list_diffs.append(probs[0] - probs[1])
        list_diffs.append(probs[0] - probs[2])

    # Plot NO
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    y3 = [val[2] for val in list_scores]
    
    width = 0.25
    
    
    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2.2, y1, width=width, label='Non-Agg')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Overtly')
    rect3 = ax.bar(x + (width+0.12), y3, width=width, label='Covertly')

    ax.set_ylabel('Probabilites')
    ax.set_xticks(x)
    ax.set_title('Probabilities Comparison Classes')
    ax.legend(loc=1)

    fig.tight_layout()
    

    plt.savefig("./plots/Probability-Confidence/Nonagg-NonMatch.png")    
    
    return list_scores, list_diffs

In [34]:
df_NG

,Unnamed: 0,Text,Pred-Label,Agg-Prob,Gen-Prob,GT-Label
0,0,sorry if i bother somebody.. iam a defence asp...,NAG-GEN,[0.81337935 0.10682596 0.07979469],[0.20062688 0.79937315],NAG-GEN
1,1,Randi Naxalist Roy,NAG-NGEN,[0.96015495 0.02288608 0.01695899],[0.9624128 0.0375873],NAG-GEN
2,2,wow.... india is progressing day by day.... No...,NAG-NGEN,[0.9327383 0.02143447 0.04582729],[0.99649435 0.00350569],NAG-GEN
3,3,ARUNDHATI ROY.....HA HA HA........HER PARENTS ...,NAG-NGEN,[0.84490246 0.03739336 0.11770418],[0.9980465 0.00195348],NAG-GEN
4,4,Allah and Muhammad can now have sex in India,NAG-NGEN,[0.90352356 0.02240491 0.07407154],[0.9886021 0.01139788],NAG-GEN
5,5,World Knows The Prime Roots of Urban NAXAL tha...,NAG-NGEN,[0.9047116 0.03067101 0.06461739],[0.99661404 0.00338593],NAG-GEN
6,6,Arundhati real name is Suzan and she is charis...,NAG-NGEN,[0.9034141 0.04429049 0.05229545],[0.9945122 0.00548781],NAG-GEN
7,7,We are here because our ancestors were against...,NAG-GEN,[0.7449015 0.08207034 0.17302823],[0.09623028 0.90376973],NAG-GEN
8,8,Homosexual hona is not a crime but it shouldn'...,NAG-NGEN,[0.95881957 0.01758637 0.02359411],[0.9765638 0.02343618],NAG-GEN
9,9,+onthebookshelf \nSexual relationships betwee...,NAG-NGEN,[0.8022871 0.05470058 0.14301236],[0.8259745 0.17402545],NAG-GEN


In [35]:
x, y = predAnalyseNoagg(df_NG)
print(x)
print(y)
np.mean(y)

[[0.04124348, 0.7354514, 0.22330508]]
[-0.69420792, -0.1820616]


-0.43813475999999996

## Gender Inference Analysis - Matching Classes 

In [2]:
# Load the Inference files
df_OG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_OG.csv")
df_CG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_CG.csv")
df_NG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_NG.csv")
df_ONG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_ONG.csv")
df_CNG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_CNG.csv")
df_NNG = pd.read_csv("/Users/amit1.bhatti/Desktop/AI-LAB/experiment/Thesis_work/TRAC-2-1/preprocessing/model_outputs/dev-prediction/pred_dev_NNG.csv")

In [67]:
def predAnalyseGenMatch(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "-GEN" in x)]
    #corr_df = corr_df[corr_df["Pred-Label"] == corr_df["GT-Label"]]
    #corr_df = df[(df["Pred-Label"].apply(lambda x : "GEN" in x)) & (df["Pred-Label"] == df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Gen-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1]])
        list_diffs.append(probs[0] - probs[1])
        
    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    
    width = 0.35

    fig, ax = plt.subplots()
    rect1 = ax.bar(x - width/2, y1, width=width, label='NonGendered')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Gendered')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Gen-Match.png")
        
    return list_scores, list_diffs

In [68]:
df_Gen = pd.concat([df_OG, df_CG, df_NG])
x, y = predAnalyseGenMatch(df_Gen)
print(len(x))
np.mean(y)

9


-0.6509483911111111

In [71]:
9/65

0.13846153846153847

In [69]:
def predAnalyseGenNMatch(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "NGEN" in x)]
    #corr_df = corr_df[corr_df["Pred-Label"] != corr_df["GT-Label"]]
    #corr_df = df[(df["Pred-Label"].apply(lambda x : "GEN" in x)) & (df["Pred-Label"] != df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Gen-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1]])
        list_diffs.append(probs[0] - probs[1])
        
    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12,7))
    rect1 = ax.bar(x - width/2, y1, width=width, label='NonGendered')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Gendered')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.set_xticklabels(x, fontsize=7)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/Gen-NonMatch.png")
        
    return list_scores, list_diffs

In [70]:
df_Gen = pd.concat([df_OG, df_CG, df_NG])
x, y = predAnalyseGenNMatch(df_Gen)
print(len(x))
np.mean(y)

56


0.8402009863008928

In [100]:
def predAnalyseNGenMatch(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "NGEN" in x)]
    #corr_df = corr_df[corr_df["Pred-Label"] == corr_df["GT-Label"]]
    # corr_df = df[(df["Pred-Label"].apply(lambda x : "NGEN" in x)) & (df["Pred-Label"] == df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Gen-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1]])
        list_diffs.append(probs[0] - probs[1])
        
    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(30,12))
    rect1 = ax.bar(x - width/2, y1, width=width, label='NonGendered')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Gendered')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/NGen-Match.png")
        
    return list_scores, list_diffs

In [101]:
df_NGen = pd.concat([df_ONG, df_CNG, df_NNG])
x, y = predAnalyseNGenMatch(df_NGen)
print(len(x))
np.mean(y)

858


0.8568549909159289

In [102]:
858/888

0.9662162162162162

In [96]:
def predAnalyseNGenNMatch(df):
    
    # Filter the Data point there the GT-Label == Pred-Label
    corr_df = df[df["Pred-Label"].apply(lambda x : "NGEN" not in x)]
    # corr_df = corr_df[corr_df["Pred-Label"] != corr_df["GT-Label"]]
    #corr_df = df[(df["Pred-Label"].apply(lambda x : "NGEN" in x)) & (df["Pred-Label"] != df["GT-Label"])]
    # For each of the datapoint, take the difference of the probability value for Overtly and Covertly
    list_scores = []
    list_diffs = []
    
    for probs in corr_df["Gen-Prob"]:
        probs = probs.replace("[", "")
        probs = probs.replace("]", "")
        probs = np.fromstring(probs, dtype=float, sep=" ")
        
        # Storage format (class_focus, target_class, class_focus - target_class)
        list_scores.append([probs[0], probs[1]])
        list_diffs.append(probs[0] - probs[1])
        
    # Plot OC
    x = np.arange(1, len(list_scores)+1)
    y1 = [val[0] for val in list_scores]
    y2 = [val[1] for val in list_scores]
    
    width = 0.35

    fig, ax = plt.subplots(figsize=(25,12))
    rect1 = ax.bar(x - width/2, y1, width=width, label='NonGendered')
    rect2 = ax.bar(x + width/2, y2, width=width, label='Gendered')

    ax.set_ylabel('Probabilites')
    ax.set_title('Probabilities Comparison Classes')
    ax.set_xticks(x)
    ax.legend(loc=1)

    fig.tight_layout()

    plt.savefig("./plots/Probability-Confidence/NGen-NMatch.png")
        
    return list_scores, list_diffs

In [97]:
x, y = predAnalyseNGenNMatch(df_NGen)
print(len(x))
np.mean(y)

30


-0.42890334599999996